In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# LOAD DATASET

In [2]:
train = pd.read_csv("./data/train.tsv", sep = "\t", index_col = "PhraseId")
train["Phrase(original)"] = train["Phrase"]
print(train.shape)
train.head(5)

(156060, 4)


,SentenceId,Phrase,Sentiment,Phrase(original)
PhraseId,,,,
1,1,A series of escapades demonstrating the adage ...,1,A series of escapades demonstrating the adage ...
2,1,A series of escapades demonstrating the adage ...,2,A series of escapades demonstrating the adage ...
3,1,A series,2,A series
4,1,A,2,A
5,1,series,2,series


In [3]:
test = pd.read_csv("./data/test.tsv", sep = "\t", index_col = "PhraseId")
test["Phrase(original)"] = test["Phrase"]
print(test.shape)
test.head(5)

(66292, 3)


,SentenceId,Phrase,Phrase(original)
PhraseId,,,
156061,8545,An intermittently pleasing but mostly routine ...,An intermittently pleasing but mostly routine ...
156062,8545,An intermittently pleasing but mostly routine ...,An intermittently pleasing but mostly routine ...
156063,8545,An,An
156064,8545,intermittently pleasing but mostly routine effort,intermittently pleasing but mostly routine effort
156065,8545,intermittently pleasing but mostly routine,intermittently pleasing but mostly routine


# PREPROCESSING

## Cleantext -- 1

NLTK(Natural Language Toolkit) provides easy-to-use interfaces to over 50 corpora and lexical resources such as WordNet, along with a suite of text processing libraries for classification, tokenization, stemming, tagging, parsing, and semantic reasoning, wrappers for industrial-strength NLP libraries, and an active discussion forum.

Snowball is a small string processing language designed for creating stemming algorithms for use in Information Retrieval.

In [4]:
import nltk
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english') 
print(stemmer.stem("disappointments")) #disappoint

disappoint


In [5]:
def stem_phrase(phrase):
    words = phrase.split(" ")
    stemmed_words = []

    for word in words:
        stemmed_word = stemmer.stem(word)
        stemmed_words.append(stemmed_word)

    stemmed_phrase = " ".join(stemmed_words)

    return stemmed_phrase

In [6]:
# Train
tqdm.pandas(desc="Stemming... (train)")

train["Phrase"] = train["Phrase"].progress_apply(stem_phrase)

print(train.shape)
train.head()

/Users/rosekoh/opt/anaconda3/envs/movie-sentiment/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
Stemming... (train): 100%|██████████| 156060/156060 [00:15<00:00, 10178.33it/s]

(156060, 4)


,SentenceId,Phrase,Sentiment,Phrase(original)
PhraseId,,,,
1,1,a seri of escapad demonstr the adag that what ...,1,A series of escapades demonstrating the adage ...
2,1,a seri of escapad demonstr the adag that what ...,2,A series of escapades demonstrating the adage ...
3,1,a seri,2,A series
4,1,a,2,A
5,1,seri,2,series


In [7]:
# Test
tqdm.pandas(desc = "Stemming... (test)")

test["Phrase"] = test["Phrase"].progress_apply(stem_phrase)

print(test.shape)
test.head()

/Users/rosekoh/opt/anaconda3/envs/movie-sentiment/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
Stemming... (test): 100%|██████████| 66292/66292 [00:05<00:00, 11416.09it/s]

(66292, 3)


,SentenceId,Phrase,Phrase(original)
PhraseId,,,
156061,8545,an intermitt pleas but most routin effort .,An intermittently pleasing but mostly routine ...
156062,8545,an intermitt pleas but most routin effort,An intermittently pleasing but mostly routine ...
156063,8545,an,An
156064,8545,intermitt pleas but most routin effort,intermittently pleasing but mostly routine effort
156065,8545,intermitt pleas but most routin,intermittently pleasing but mostly routine


## Cleantext 2.1

In [8]:
from wordcloud import WordCloud, STOPWORDS

In [9]:
# nltk.download("stopwords")
stopwords = nltk.corpus.stopwords.words('english')
# extra_stopwords = []
# stopwords.extend(extra_stopwords)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rosekoh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
# wordcloud color
def random_color_func(word=None, font_size=None, position=None,  
                      orientation=None, font_path=None, random_state=None):
    h = int(360.0 * float(random_state.randint(100, 150) / 255.0))
    s = int(73)
    l = int(48)

    return "hsl({}, {}%, {}%)".format(h, s, l)

In [11]:
import matplotlib.pyplot as plt
%matplotlib inline
wordcloud = WordCloud(stopwords = stopwords,
                      background_color = 'white',
                      width = 800, height = 600,
                      color_func=random_color_func).generate(' '.join(train['Phrase']))

plt.figure(figsize = (15, 10))
plt.axis("off")
plt.imshow(wordcloud)
plt.show()

## Cleantext -- 2.2

In [12]:
def clean_text(phrase):
    phrase = phrase.replace("n't", "not")
    phrase = phrase.replace("not", "no")
    phrase = phrase.replace("hopeless", "bad")
    phrase = phrase.replace("good", "best")
    phrase = phrase.replace("excellent", "best")
    phrase = phrase.replace("funni", "fun")
    phrase = phrase.replace("funny", "fun")
    phrase = phrase.replace("littl", "little")
    phrase = phrase.replace("the movi", "movie")
    phrase = phrase.replace("veri", "very")
    phrase = phrase.replace("onli", "only")
    phrase = phrase.replace("comedi", "comedy")
    phrase = phrase.replace("veri", "very")
    phrase = phrase.replace("stori", "story")
    phrase = phrase.replace("charact", "character")

    return phrase

train["Phrase"] = train["Phrase"].apply(clean_text)
test["Phrase"] = test["Phrase"].apply(clean_text)

In [13]:
import matplotlib.pyplot as plt
%matplotlib inline
wordcloud = WordCloud(stopwords = stopwords,
                      background_color = 'white',
                      width = 800, height = 600,
                      color_func=random_color_func).generate(' '.join(train['Phrase']))

plt.figure(figsize = (15, 10))
plt.axis("off") # 축에 표시되는 눈금을 제거하는 옵션
plt.imshow(wordcloud) # 이미지가 표시되도록 하는 옵션.
plt.show() # 최종으로 보여주는 옵션.

### one Hot Encode Phrases

**TF-IDF**
$$ \frac{TF}{IDF} $$

* Term frequency  (per sentence)
* Document frequency  (per document)

values uniqueness 1 > 0 (a,the,etc..)
* tree(countVectorizer)
* regression(TFIDF)

When it comes to NLP, even if it is just one thing that changed, tuning has to be re-done such as learning rate alpha.

#### CountVectorizer

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features = 30000, ngram_range = (1,4), stop_words =['the',
'of','to','it','in','that','an','of the','this','his','about','at','or','than','from','in the','are','so','rrb',
'the film','who','lrb','to the','doe','do','for the','director','been','ani','on the'])

vectorizer.fit(train["Phrase"])

/Users/rosekoh/opt/anaconda3/envs/movie-sentiment/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['film', 'for', 'on'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=30000, min_df=1,
                ngram_range=(1, 4), preprocessor=None,
                stop_words=['the', 'of', 'to', 'it', 'in', 'that', 'an',
                            'of the', 'this', 'his', 'about', 'at', 'or',
                            'than', 'from', 'in the', 'are', 'so', 'rrb',
                            'the film', 'who', 'lrb', 'to the', 'doe', 'do',
                            'for the', 'director', 'been', 'ani', 'on the'],
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [15]:
X_train = vectorizer.transform(train["Phrase"])
# applies the params from fiting to data

print(X_train.shape)
X_train

(156060, 30000)


<156060x30000 sparse matrix of type '<class 'numpy.int64'>'
	with 1197794 stored elements in Compressed Sparse Row format>

In [16]:
columns = vectorizer.get_feature_names()# <- vocab
# "not funny" in columns

train_vector = pd.DataFrame(X_train.toarray(), columns= columns)
print(train_vector.shape)

(156060, 30000)


In [17]:
# Visualize

import matplotlib.pyplot as plt

dictionary = {}

# This takes a while
# This will take each train dataset's word and count the frequency in dictionary form.
for column in train_vector.columns:
    dictionary[column] = train_vector[column].sum()

dictionary_dataframe = pd.DataFrame.from_dict({'word': list(dictionary.keys()), 'count': list(dictionary.values())})
dictionary_dataframe = dictionary_dataframe.sort_values('count', ascending=False)
dictionary_dataframe = dictionary_dataframe[0:100]

In [18]:
%matplotlib inline
import seaborn as sns

figure, ax = plt.subplots(nrows=1, ncols=1)
figure.set_size_inches(18, 64)

sns.barplot(data=dictionary_dataframe, y="word", x="count", ax=ax)
# dictionary_dataframe.to_csv("frequency.csv")

#### TfidfVectorizer

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

char_vectorizer = TfidfVectorizer(analyzer = 'char', 
                                  max_features = 10000, 
                                  ngram_range = (1,9))

char_vectorizer.fit(train["Phrase"])
%time

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 4.77 µs


In [20]:
word_vectorizer = TfidfVectorizer(analyzer = 'word', 
                                  max_features = 30000, 
                                  ngram_range = (1,4))

word_vectorizer.fit(train["Phrase"])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=30000,
                min_df=1, ngram_range=(1, 4), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

# Fit and Transform 

In [21]:
X_train_word = word_vectorizer.transform(train["Phrase"])
print(X_train_word.shape)
X_train_char = char_vectorizer.transform(train["Phrase"])
print(X_train_char.shape)

from scipy.sparse import hstack #vstack, hstack (word, character to merge )
X_train = hstack([X_train_char, X_train_word])
print(X_train.shape)
X_train

(156060, 30000)
(156060, 10000)
(156060, 40000)


<156060x40000 sparse matrix of type '<class 'numpy.float64'>'
	with 19680572 stored elements in COOrdinate format>

In [22]:
X_test_word = word_vectorizer.transform(test["Phrase"])
print(X_test_word.shape)
X_test_char = char_vectorizer.transform(test["Phrase"])
print(X_test_char.shape)

X_test = hstack([X_test_char, X_test_word])
print(X_test.shape)
X_test

(66292, 30000)
(66292, 10000)
(66292, 40000)


<66292x40000 sparse matrix of type '<class 'numpy.float64'>'
	with 7612200 stored elements in COOrdinate format>

In [23]:
y_train = train["Sentiment"]

print(y_train.shape)
y_train.head()

(156060,)


PhraseId
1    1
2    2
3    2
4    2
5    2
Name: Sentiment, dtype: int64

# Score 

In [24]:
from sklearn.linear_model import SGDClassifier

seed = 23

model = SGDClassifier(n_jobs=-1,
                      alpha = 0.00005,
                      random_state=seed)

model

SGDClassifier(alpha=5e-05, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=-1, penalty='l2',
              power_t=0.5, random_state=23, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [25]:
#cross validation

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score

predictions = cross_val_predict(model, X_train, y_train, cv = 5)

score = accuracy_score(y_train, predictions)

print("Score = {0:.5f}".format(score)) #0.59162

Score = 0.59162


In [26]:
from sklearn.model_selection import cross_val_score, GroupKFold
# from sklearn.cross_validation import cross_val_score, GroupKFold

kfold = GroupKFold(n_splits=5)

score = cross_val_score(model, X_train, y_train, cv=kfold, groups=train["SentenceId"]).mean()

print("Score = {0:.5f}".format(score)) #0.59840

Score = 0.59840


In [27]:
import xgboost as xgb

seed = 23

dtrain = xgb.DMatrix(X_train, label = y_train)

params = {
    'booster': 'gblinear',
    'objective': 'multi:softmax',
    'lambda': 2.186753e-03,
    'alpha': 1.286904,
    'lambda_bias': 6.191707e+00,
    'num_class': 5,
    'nthread':2,
}

booster= xgb.train(params, dtrain, num_boost_round=98)
booster

# Predictions

In [28]:
dtest = xgb.DMatrix(X_test.toarray())

In [29]:
predictions = booster.predict(dtest)
print(predictions.shape)
predictions[0:10]

(66292,)


array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2.], dtype=float32)

In [30]:
model.fit(X_train, y_train)
predictions = model.predict(X_test)
print(predictions.shape)
predictions[0:10]

(66292,)


array([3, 3, 2, 3, 3, 3, 3, 2, 3, 2])

# Submit

In [31]:
submission = pd.read_csv("./data/sampleSubmission.csv", index_col="PhraseId")

submission["Sentiment"] = predictions

print(submission.shape)
submission.head()

(66292, 1)


,Sentiment
PhraseId,
156061,3
156062,3
156063,2
156064,3
156065,3


In [32]:
from datetime import datetime

current_date = datetime.now()
current_date = current_date.strftime("%Y-%m-%d_%H-%M-%S")

desc = ""
sign = "silverrain"

filename = "{date}_{score:.5f}_{desc}_{sign}.csv".format(date=current_date, 
                                                         score=score, 
                                                         desc=desc, 
                                                         sign=sign)
filepath = "{filename}".format(filename=filename)

In [33]:
submission.to_csv(filepath)